In [1]:
import networkx as nx
import os
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import csv
import copy
from scipy.optimize import curve_fit

def mk_int(s):
    s = s.strip()
    return int(s) if s else 0

In [2]:
# These are the values to be read from the csv file
ports = ['1', '2', '3', '4']
years = ['2005', '2010', '2015', '2017']
codes = ['2701', '2709', '2711', '271111', '271121']

# Initialising the dictionary with network for each combinations
G = {(port, year, code): nx.DiGraph() for port in ports[:2] for year in years for code in codes}

# Column numbers for reading from and to node from the csv file
# 3 - ReExports have to be substracted from Exports
# 4 - ReImports have to be substracted from Imports
dictPort = {'1': (12, 9),
            '2': (9, 12),
            '3': (9, 12),
            '4': (12, 9)
           }
# Unit conversion for each type of fuel - kg to TJ
dictWeight = {'2701': 25750 * 10 ** (-9),  
              '2709': 45012.78 * 10 ** (-9), 
              '271111': 48960 * 10 ** (-9), 
              '271121': 45860 * 10 ** (-9)
             }

# Column numbers for getting these data from the csv file
codeIndex = 21
yearIndex = 1
portIndex = 6

# Saving code numbers for fuels in variables
petrolCode = '271111'
gasCode = '271121'
genCode = '2711'

#looping through all csv files in this directory
for file in os.listdir('PycharmProjects/EnergyTradeAnalysis/Database/Data_All'):
    with open('PycharmProjects/EnergyTradeAnalysis/Database/Data_All/' + file, encoding='utf-8', errors='ignore') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)
        for row in csv_reader:
            for port in ports[:2]:
                #print("Checking in port: ", row[portIndex], port)
                if row[portIndex] == port:
                    for year in years:
                        #print("Checking in year: ", row[yearIndex])
                        if row[yearIndex] == year:
                            for code in codes:
                                #print("Checking in code: ", row[codeIndex])
                                if row[codeIndex] == code:
                                    endpts = dictPort[port]
                                    points = [row[endpts[0]], row[endpts[1]]]
                                    weightInKJ = dictWeight[code] * mk_int(row[len(row) - 10])
                                    G[port, year, code].add_edge(points[0], points[1], weight=weightInKJ, cost=row[len(row) - 4])
                                    if code == petrolCode or code == gasCode:
                                        if not G[(port, year, genCode)].has_edge(points[0], points[1]):
                                            G[(port, year, genCode)].add_edge(points[0], points[1], weight=weightInKJ, cost=row[len(row) - 4])
                                        else:
                                            if code == petrolCode:
                                                if G[(port, year, gasCode)].has_edge(points[0], points[1]):
                                                    weight = G[(port, year, gasCode)][points[0]][points[1]]['weight'] 
                                                    cost = G[(port, year, gasCode)][points[0]][points[1]]['cost']
                                                    G[(port, year, genCode)][points[0]][points[1]]['weight'] += G[(port, year, petrolCode)][points[0]][points[1]]['weight'] 
                                                    G[(port, year, genCode)][points[0]][points[1]]['cost'] += G[(port, year, petrolCode)][points[0]][points[1]]['cost']
                                            else:
                                                if G[(port, year, petrolCode)].has_edge(points[0], points[1]):
                                                    weight = G[(port, year, petrolCode)][points[0]][points[1]]['weight'] 
                                                    cost = G[(port, year, petrolCode)][points[0]][points[1]]['cost']
                                                    G[(port, year, genCode)][points[0]][points[1]]['weight'] += G[(port, year, gasCode)][points[0]][points[1]]['weight'] 
                                                    G[(port, year, genCode)][points[0]][points[1]]['cost'] += G[(port, year, gasCode)][points[0]][points[1]]['cost']
                                                    

# This second loop is to take care of the reExport and reImport cases in our csv files 

for file in os.listdir('PycharmProjects/EnergyTradeAnalysis/Database/Data_All'):
    with open('PycharmProjects/EnergyTradeAnalysis/Database/Data_All/' + file, encoding='utf-8', errors='ignore') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)
        for row in csv_reader:
            if row[portIndex] in ports[2:]:
                for port in ports[2:]:
                    #print("Checking in port: ", row[portIndex], port)
                    if row[portIndex] == port:
                        for year in years:
                            #print("Checking in year: ", row[yearIndex])
                            if row[yearIndex] == year:
                                for code in codes:
                                    #print("Checking in code: ", row[codeIndex])
                                    if row[codeIndex] == code:
                                        endpts = dictPort[port]
                                        points = [row[endpts[0]], row[endpts[1]]]
                                        # Check if it is re-export case
                                        if port == ports[2]:
                                            # Check if it is coal or crude oil
                                            if code in codes[:2]:
                                                if G[(ports[1], year, code)].has_edge(points[0], points[1]):
                                                    weightInKJ = dictWeight[code] * mk_int(row[len(row) - 10])
                                                    G[(ports[1], year, code)][points[0]][points[1]]['weight'] += -1 * weightInKJ
                                            # cases of lpg liquid and lpg gas
                                            else:
                                                if G[(ports[1], year, code)].has_edge(points[0], points[1]):
                                                    weightInKJ = dictWeight[code] * mk_int(row[len(row) - 10])
                                                    G[(ports[1], year, code)][points[0]][points[1]]['weight'] += -1 * weightInKJ
                                                if G[(ports[1], year, genCode)].has_edge(points[0], points[1]):
                                                    weightInKJ = dictWeight[code] * mk_int(row[len(row) - 10])
                                                    G[(ports[1], year, genCode)][points[0]][points[1]]['weight'] += -1 * weightInKJ
                                                
                                        # Check if it is re-import case
                                        else:
                                            # Check if it is coal or crude oil
                                            if code in codes[:2]:
                                                if G[(ports[0], year, code)].has_edge(points[0], points[1]):
                                                    weightInKJ = dictWeight[code] * mk_int(row[len(row) - 10])
                                                    G[(ports[0], year, code)][points[0]][points[1]]['weight'] += -1 * weightInKJ
                                            
                                            # cases of lpg liquid and lpg gas
                                            else:
                                                if G[(ports[0], year, code)].has_edge(points[0], points[1]):
                                                    weightInKJ = dictWeight[code] * mk_int(row[len(row) - 10])
                                                    G[(ports[0], year, code)][points[0]][points[1]]['weight'] += -1 * weightInKJ
                                                if G[(ports[0], year, genCode)].has_edge(points[0], points[1]):
                                                    weightInKJ = dictWeight[code] * mk_int(row[len(row) - 10])
                                                    G[(ports[0], year, genCode)][points[0]][points[1]]['weight'] += -1 * weightInKJ                          

In [3]:
# This cell is just to get the list of total nodes and unique nodes in all our networks together

nodeList = []
for year in years:
    for port in ports[:2]:
        for code in codes[:3]:
            nodeList += list(G[port, year, code].nodes())

print("totalNodes: ", len(nodeList))
uniqueNodes = [] 
for i in nodeList: 
    if i not in uniqueNodes: 
        uniqueNodes.append(i)
print("uniqueNodes: ", len(uniqueNodes))

totalNodes:  4104
uniqueNodes:  244


In [4]:
# This cell is to print the list of unique nodes in all our networks together

with open('PycharmProjects/EnergyTradeAnalysis/Results/UNData/UniqueCountries.csv','w') as csv_file:
    writer = csv.writer(csv_file)
    for node in uniqueNodes:
        writer.writerow([node])

In [5]:
# In this cell, we delete all these nodes and the edges associted with these nodes

toDeleteNodes = ['Areas, nes','ASEAN','EU-28','Free Zones','LAIA, nes',
                 'North America and Central America, nes','Oceania, nes','Other Asia, nes',
                 'Other Europe, nes','Special Categories','World','Other Africa, nes', 'Bunkers', 
                 'Serbia and Montenegro']
for year in years:
    for code in codes:
        for port in ports[:2]:
            G[port, year, code].remove_nodes_from(toDeleteNodes)

In [6]:
# This matrix dictionary holds the sorted adjacency matrices, 
# along with row sum and column sum in each matrix as DataFrames 
matrix = {}
for graphKey in G:
    nodeList = sorted(list(G[graphKey].nodes()))
    matrix[graphKey] = nx.to_pandas_adjacency(G[graphKey], nodelist=nodeList, weight='weight')
    matrix[graphKey]["Sum"] = matrix[graphKey].sum(axis=1)
    matrix[graphKey].loc["Sum"] = matrix[graphKey].sum()
    matrix[graphKey].to_csv('PycharmProjects/EnergyTradeAnalysis/Results/AdjacencyWeightUpdated/AdjWeight_' + str(graphKey) + '.csv', mode = 'w')

In [18]:
GUnion = {(year, code): nx.DiGraph() for year in years for code in codes[:3]}

# After this loop GUnion only has network created by import matrix data
for year in years:
    for code in codes[:3]:
        GUnion[(year, code)] = copy.deepcopy(G[(ports[0], year, code)])

# In this loop, we add new edges from export matrix to GUnion and 
# if the weight value in import matrix is 0, we update it with weight value from export matrix for that edge and
# if the weight in import matrix is different from export matrix, we take the mean of it
for year in years:
    for code in codes[:3]:
        # for each edge in export network
        for edge in list(G[(ports[1], year, code)].edges(data=True)):
            # if the weight of the edge from export network is above 2 TJ (i.e significant)
            if G[(ports[1], year, code)][edge[0]][edge[1]]['weight'] > 2:
                # if that particular edge is not availble in GUnion we add it
                if not GUnion[(year, code)].has_edge(edge[0], edge[1]):
                    wt = G[(ports[1], year, code)][edge[0]][edge[1]]['weight']
                    ct = G[(ports[1], year, code)][edge[0]][edge[1]]['cost']                    
                    GUnion[(year, code)].add_edge(edge[0], edge[1], weight=wt, cost=ct)
                # if particular edge is avaialble in GUnion, we check its weight
                else:
                    # if the weight in GUnion <= 2  (i.e not significant), we update it with the value from export network
                    if GUnion[(year, code)][edge[0]][edge[1]]['weight'] <= 2:
                        GUnion[(year, code)][edge[0]][edge[1]]['weight'] = G[(ports[1], year, code)][edge[0]][edge[1]]['weight']                        
                    else:
                        meanWt = (G[(ports[0], year, code)][edge[0]][edge[1]]['weight'] + G[(ports[1], year, code)][edge[0]][edge[1]]['weight']) / 2
                        GUnion[(year, code)][edge[0]][edge[1]]['weight'] = meanWt

In [19]:
#Updating the data from IEA, as UN had wrong data

Country1 = 'USA'
Country2 = 'Mexico'

gasUSAToMex = {
                '2005': 357481, 
                '2010': 372182,
                '2015': 1142431,
                '2017': 1813170
              } 
gasMexToUSA = {
                '2005': 9834, 
                '2010': 31648,
                '2015': 984,
                '2017': 1420
              } 

for year in years:
    for edge in list(GUnion[(year, genCode)].edges(data=True)):
        if edge[0] == Country1 and edge[1] == Country2:
            print(year, genCode, ":")
            GUnion[(year, genCode)][edge[0]][edge[1]]['weight'] = gasUSAToMex[year]
            print(edge) 
        elif edge[0] == Country2 and edge[1] == Country1:
            print(year, genCode, ":")
            GUnion[(year, genCode)][edge[0]][edge[1]]['weight'] = gasMexToUSA[year]
            print(edge)

2005 2711 :
('USA', 'Mexico', {'weight': 357481, 'cost': '4410907372255509584'})
2005 2711 :
('Mexico', 'USA', {'weight': 9834, 'cost': '253288640978654'})
2010 2711 :
('USA', 'Mexico', {'weight': 372182, 'cost': '1561851711363688170'})
2010 2711 :
('Mexico', 'USA', {'weight': 31648, 'cost': '11787432747573'})
2015 2711 :
('USA', 'Mexico', {'weight': 1142431, 'cost': '1177406552958260884'})
2015 2711 :
('Mexico', 'USA', {'weight': 984, 'cost': '15806814'})
2017 2711 :
('USA', 'Mexico', {'weight': 1813170, 'cost': '9627960624857807688'})
2017 2711 :
('Mexico', 'USA', {'weight': 1420, 'cost': '12326030'})


In [20]:
# This matrixUnion dictionary holds the sorted adjacency matrices, 
# along with row sum and column sum in each matrix as DataFrames for all edges 

matrixUnion = {}
for graphKey in GUnion:
    nodeList = sorted(list(GUnion[graphKey].nodes()))
    matrixUnion[graphKey] = nx.to_pandas_adjacency(GUnion[graphKey], nodelist=nodeList, weight='weight')
    matrixUnion[graphKey]["Sum"] = matrixUnion[graphKey].sum(axis=1)
    matrixUnion[graphKey].loc["Sum"] = matrixUnion[graphKey].sum()
    matrixUnion[graphKey].to_csv('PycharmProjects/EnergyTradeAnalysis/Results/AdjacencyWeightUnion/AdjWeight_' + str(graphKey) + '.csv', mode = 'w')